In [1]:
import sys
sys.path.append('..')

In [2]:
from agents.dqn import *
from core.replay import *
from environments.wrappers import *
from networks.nature_cnn import *
from utils.train import *
from utils.logger import *

In [3]:
import gym
import torch

### Network, Environment, Agent

In [4]:
network_params = [
            torch.nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            torch.nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        ]

In [5]:
# parameters
size = (84, 84)
num_stacked_frames = 4

raw_env = gym.make('BreakoutNoFrameskip-v4')
env = AtariWrapper(raw_env, size=size, k=num_stacked_frames)

In [6]:
observation_space = raw_env.observation_space
action_space = raw_env.action_space

params = {'epsilon':1.0, 'epsilon_decay':0.9999, 'epsilon_min':0.1, 'gamma':0.99, 'alpha':2.5e-5, 'network_params': network_params,
          'memory_size':200000, 'device':'cuda:0', 'batch_size':32, 'target_net_updates':10000}

agent = DQNAgent(observation_space, action_space, **params)

In [7]:
agent.network

DQNCNN(
  (network): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=512, bias=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=4, bias=True)
  )
)

### Standard Train

In [8]:
logger = Logger('training_info')

In [9]:
save_dir = './models/breakout/'
training_params = {'num_steps': 0, 'total_steps':20000000, 'logger':logger, 'save_freq':1000, 'e_verbose':100, 'file_name': 'atari ddqn', 'save_dir':save_dir}

In [10]:
# standard_train(agent, env, **training_params)

### Runner Train

In [11]:
save_dir = './models/breakout/'
training_params = {'num_steps': 0, 'total_steps':20000000, 'logger':logger, 't_save_freq':50000, 't_verbose':50000, 'file_name': 'atari ddqn', 'save_dir':save_dir,
                   'steps': 16}

In [12]:
runner_train(agent, env, **training_params)

AttributeError: 'numpy.ndarray' object has no attribute 'append'